In [ ]:
from winevt import EventLog

events = []

def handleEvent(action, pContext, event):
    evt = {}
    if hasattr(event, "EventData"):
        if hasattr(event.EventData, "Data"):
            if event.EventData.Data[1].cdata == "Furqan Saeed":
                evt["EventID"] = event.EventID
                for item in event.EventData.Data:
                    evt[item["Name"]] = item.cdata
                events.append(evt)

cb = EventLog.Subscribe("Security", "Event/System[EventID=4663 or EventID=4660]", handleEvent)

In [ ]:
import win32evtlog

num = 0;
hand=win32evtlog.OpenEventLog(None, "Security")
flags= win32evtlog.EVENTLOG_BACKWARDS_READ|win32evtlog.EVENTLOG_SEQUENTIAL_READ

while(1):
    records = win32evtlog.ReadEventLog(hand, flags, 0)
    # print(dir(records[0]))
    if not records:
        break;
    for record in records:
        if record.EventID == 4663:
            print(str(record.EventID) + " " + record.SourceName + " " + str(record.ComputerName) + " " + record.TimeWritten.Format())
            num = num + 1
    
win32evtlog.CloseEventLog(hand)
print(num)

In [ ]:
import os
import sys, signal
import win32file
import win32con

def signal_handler(signal, frame):
    print("\nprogram exiting gracefully")
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

ACTIONS = {
    1: "Created",
    2: "Deleted",
    3: "Updated",
    4: "Renamed to ",
    5: ""
}
FILE_LIST_DIRECTORY = 0x0001

path_to_watch = "D:/SIEM/test"

hDir = win32file.CreateFile(
    path_to_watch,
    FILE_LIST_DIRECTORY,
    win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE | win32con.FILE_SHARE_DELETE,
    None,
    win32con.OPEN_EXISTING,
    win32con.FILE_FLAG_BACKUP_SEMANTICS,
    None
)

while 1:

    results = win32file.ReadDirectoryChangesW(
        hDir,
        1024,
        True,
        win32con.FILE_NOTIFY_CHANGE_FILE_NAME |
        win32con.FILE_NOTIFY_CHANGE_DIR_NAME |
        # win32con.FILE_NOTIFY_CHANGE_ATTRIBUTES |
        # win32con.FILE_NOTIFY_CHANGE_SIZE |
        win32con.FILE_NOTIFY_CHANGE_LAST_WRITE,
        # win32con.FILE_NOTIFY_CHANGE_SECURITY,
        None,
        None
    )

    for action, file in results:
        full_filename = os.path.join(path_to_watch, file)
        if action == 4:
            print(full_filename, ACTIONS.get(action, "Unknown"), end='')
        else :
            print(full_filename,ACTIONS.get(action, "Unknown"))

In [ ]:
import os
import threading
import sys, signal
import win32file
import win32con
from dataclasses import dataclass
from typing import Callable, List
import subprocess
import json
import time
import datetime


def signal_handler(signal, frame):
    t1.join()
    t2.join()
    t3.join()
    print("\nprogram exiting gracefully")
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

ACTIONS = {
    1: "Created",
    2: "Deleted",
    3: "Updated",
    4: "Renamed to ",
    5: ""
}

diskFiles = []
folderFiles = []

FILE_LIST_DIRECTORY = 0x0001

def watchFile(dirname, removable_disk = False):
    print(dirname)
    path_to_watch = dirname

    hDir = win32file.CreateFile(
        path_to_watch,
        FILE_LIST_DIRECTORY,
        win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE | win32con.FILE_SHARE_DELETE,
        None,
        win32con.OPEN_EXISTING,
        win32con.FILE_FLAG_BACKUP_SEMANTICS,
        None
    )
    while 1:

        results = win32file.ReadDirectoryChangesW(
            hDir,
            1024,
            True,
            win32con.FILE_NOTIFY_CHANGE_FILE_NAME |
            win32con.FILE_NOTIFY_CHANGE_DIR_NAME |
            # win32con.FILE_NOTIFY_CHANGE_ATTRIBUTES |
            # win32con.FILE_NOTIFY_CHANGE_SIZE |
            win32con.FILE_NOTIFY_CHANGE_LAST_WRITE,
            # win32con.FILE_NOTIFY_CHANGE_SECURITY,
            None,
            None
        )

        for action, file in results:
            full_filename = os.path.join(path_to_watch, file)
            if action == 4:
                print(full_filename, ACTIONS.get(action, "Unknown"), end='')
            else :
                print(full_filename,ACTIONS.get(action, "Unknown"))
                if action != 5:
                    if removable_disk:
                        diskFiles.append([full_filename, action, time.time()])
                    else:
                        folderFiles.append([full_filename, action, time.time()])

@dataclass
class Drive:
    letter: str
    label: str
    drive_type: str

def list_drives() -> List[Drive]:
    
    proc = subprocess.run(
        args=[
            'powershell',
            '-noprofile',
            '-command',
            'Get-WmiObject -Class Win32_LogicalDisk | Select-Object deviceid,volumename,drivetype | ConvertTo-Json'
        ],
        text=True,
        stdout=subprocess.PIPE
    )
    if proc.returncode != 0 or not proc.stdout.strip():
        print('Failed to enumerate drives')
        return []
        
    devices = json.loads(proc.stdout)

    drive_types = {
        0: 'Unknown',
        1: 'No Root Directory',
        2: 'Removable Disk',
        3: 'Local Disk',
        4: 'Network Drive',
        5: 'Compact Disc',
        6: 'RAM Disk',
    }

    removables = []

    for d in devices:
        if d['drivetype'] == 2:
            removables.append(Drive(
                letter = d['deviceid'],
                label = d['volumename'],
                drive_type = drive_types[d['drivetype']]
            ))
    
    return removables

def check_device_change(old_devices):
    devices = list_drives()
    count = len(devices) - len(old_devices)
    t2 = None

    if count > 0:
        print('added: {}'.format(devices[0]))
        t2 = threading.Thread(target = watchFile, args=(devices[0].letter +"/", True,))
        t2.start()

    if count < 0:
        print('removed: {}'.format(old_devices[0]))
        if t2 is not None:
            t2.join()
            t2 = None
        
    return devices

def removable_storage_listener():
    old_devices = []
    while(1):
        old_devices = check_device_change(old_devices)
        
        time.sleep(1)

def take_time(elem):
    return elem[2]

def get_filename(full_name):
    head, tail = os.path.split(full_name)
    return tail

if __name__ == '__main__':
    try:
        t1 = threading.Thread(target = watchFile, args=("D:/SIEM/test1",))
        t3 = threading.Thread(target = removable_storage_listener, args=())
        t1.start()
        t3.start()

        while 1:
            diskFiles.sort(key=take_time, reverse=True)
            folderFiles.sort(key=take_time, reverse=True)

            for r_dir in diskFiles:
                for f_dir in folderFiles:
                    
                    if r_dir[2] >= f_dir[2] - 5 and r_dir[2] <= f_dir[2] + 5:

                        if r_dir[1] == 1 and f_dir[1] == 2:
                            if get_filename(r_dir[0]) == get_filename(f_dir[0]):
                                print(r_dir)
                                print(f_dir)
                
                

            time.sleep(1)

    except KeyboardInterrupt:
        print("Interrupted")
        t1.join()
        t2.join()
        t3.join()
        sys.exit(0)

In [ ]:
from dataclasses import dataclass
from typing import Callable, List
import subprocess
import json
import time

@dataclass
class Drive:
    letter: str
    label: str
    drive_type: str

def list_drives() -> List[Drive]:
    
    proc = subprocess.run(
        args=[
            'powershell',
            '-noprofile',
            '-command',
            'Get-WmiObject -Class Win32_LogicalDisk | Select-Object deviceid,volumename,drivetype | ConvertTo-Json'
        ],
        text=True,
        stdout=subprocess.PIPE
    )
    if proc.returncode != 0 or not proc.stdout.strip():
        print('Failed to enumerate drives')
        return []
        
    devices = json.loads(proc.stdout)

    drive_types = {
        0: 'Unknown',
        1: 'No Root Directory',
        2: 'Removable Disk',
        3: 'Local Disk',
        4: 'Network Drive',
        5: 'Compact Disc',
        6: 'RAM Disk',
    }

    removables = []

    for d in devices:
        if d['drivetype'] == 2:
            removables.append(Drive(
                letter=d['deviceid'],
                label=d['volumename'],
                drive_type=drive_types[d['drivetype']]
            ))
    
    return removables

def check_device_change(old_devices):
    devices = list_drives()
    count = len(devices) - len(old_devices)
    if count > 0:
        print('added: {}'.format(devices[0]))
    if count < 0:
        print('removed: {}'.format(old_devices[0]))
    return devices

if __name__ == '__main__':
    try:
        old_devices = list_drives()
        while(1):
            old_devices = check_device_change(old_devices)
            
            time.sleep(1)  

    except KeyboardInterrupt:
        print("Interrupted")
        sys.exit(0)

In [ ]:
import time
import serial.tools.list_ports # pyserial

def enumerate_serial_devices():
    return set([item
        for item in list_ports.comports()
    ])

def check_new_devices(old_devices):
    devices = enumerate_serial_devices()
    added = devices.difference(old_devices)
    removed = old_devices.difference(devices)
    if added:
        print('added: {}'.format(added))
    if removed:
        print('removed: {}'.format(removed))
    return devices

# Quick and dirty timing loop
old_devices = enumerate_serial_devices()

while True:
   old_devices = check_new_devices(old_devices)
time.sleep(0.5)